# Task 2

### Training a Variational Circuit to transform random input states into predefined output states.

(using pennylane)

In [1]:
import os

try:
  import pennylane as qml
  from pennylane import numpy as np
  from pennylane.optimize import NesterovMomentumOptimizer

except:
  os.system("pip install pennylane")
  os.system("pip install torch")
  import pennylane as qml
  from pennylane import numpy as np
  from pennylane.optimize import NesterovMomentumOptimizer


Target states given:
1. |0011>
2. |0101>
3. |1010>
4. |1100>

Random input states taken:
1. |0111>
2. |1101>
3. |0000>
4. |1001>

---



In [2]:
input_states = ["0111", "1101", "0000", "1001"]
target_states = ["0011", "0101", "1010", "1100"]

n_qubits = 4
np.random.seed(0)

### Setting up constants

We start by setting up some tweakable values for the QML algorithm.

After testing out by trial and error a couple of times (and some intuition), these tweakable constants were fine for a good accuracy:



In [3]:
# Tweakable Constants

n_layers = 3
steps = 200
stepsize = 0.01
momentum=0.7

In [4]:
# Some Helper functions

# convert a string state into a list state
def string_to_list(qubits):
  parsed_list = []
  for q in qubits:
    parsed_list.append(1 if q == "1" else 0)
  return parsed_list

# convert the obtained expected value into a string
def prediction_to_string(pred):
  s = ""
  for p in pred:
    s += "1" if p < 0 else "0"
  return s

# convert the target states into the format appropriate for comparing with the expected values obtained
def target_to_exp(state_id):
  l = string_to_list(target_states[state_id])
  return np.array([-1 if l[i] == 1 else 1 for i in range(len(target_states))])

### Circuit Construction

The Circuit for our Variational Circuit Training algorithm consists of multiple layers.

Each layer is made of RX, RY and RZ gates for each qubit (with variable arguments) followed by all the qubits CNOT'ed with each other.

<b> Why is the construction like this? </b>

We want to give our input state maximum freedom to transform into the target state.

The RX, RY and RZ gates provide the ability to rotate arbitrarily along any axis. The CNOT gates make sure that the qubits are entangled with each other. 

These two things give freedom to the qubits to tranform however needed.

In addition to this, having multiple layers gives even more freedom to get the required transformation.

In [5]:
# Circuit Construction

def put_layers(weights):

  for layer in range(n_layers):

    for q in range(n_qubits):
      # rotate arbitrarily along any axis
      qml.RX(weights[layer, q, 0], wires=q)
      qml.RY(weights[layer, q, 1], wires=q)
      qml.RZ(weights[layer, q, 2], wires=q)
    
    for q in range(n_qubits):
      qml.CNOT(wires=[q % n_qubits, (q+1) % n_qubits])

# prepare the inital input state from string
def prepare_states(state_id):
  state = input_states[state_id]
  qml.BasisState(np.array(string_to_list(state)), wires=[i for i in range(n_qubits)])

After setting up appropriate gates, we find and return the expected value of the 4 qubits (using the PauliZ observable, of course). 


In [6]:
# The actual Quantum Circuit

dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(weights, state_id):

  prepare_states(state_id)
  put_layers(weights)

  # measure and return expected value
  return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

An appropriate cost function is required by the optimizer.

For our case, we will use the square mean loss function as defined below:

(Also a bias is present to provide better optimization)

<b>Why the square loss function?</b>

An alternative would be to take the sum of absolute values of the components of the difference vector.

But squaring is more useful as it supresses small losses and amplifies bigger losses.

In [7]:
# The cost function

def cost(params):
  weights = params[0]
  bias = params[1]
  
  total_cost = 0
  
  for state_id in range(len(input_states)):
    exp = np.array(circuit(weights, state_id)) + bias
    total_cost += np.mean( (target_to_exp(state_id) - exp) ** 2 )
  
  return total_cost

In [8]:
# Accuracy

def accuracy(predictions):
  total = len(target_states)
  loss = 0

  for state_id in range(len(target_states)):

    if prediction_to_string(predictions[state_id]) != target_states[state_id]:
      loss += 1
  
  return 1 - loss/total

In [9]:
# Final Circuit Tester

def circuit_tester(weights, target=True):
  print("Transformations done:\n(input -> output)\n")

  for state_id in range(len(input_states)):
    prediction = circuit(weights, state_id)
    print(f"{input_states[state_id]} -> {prediction_to_string(prediction)}")
    if target:
      print(f"target: {target_states[state_id]}\n")

Now that we have setup all the required functions, we initialise the optimizer.

We use the NesterovMomentumOptimizer which is a gradient-descent optimizer with Nesterov momentum.

We also define the parameters (initially random) that will be optimised later.

In [10]:
# Optimisation initialisation

opt = NesterovMomentumOptimizer(stepsize=stepsize, momentum=momentum)
params = (0.01 * np.random.randn(n_layers, n_qubits, 3), 0.0)

The optimiser takes steps and during each step, optimises the values of our parameters using gradient descent such that the cost is minimised. 

With initially random parameters, there is a high chance that we get 0 accuracy on the first few steps. Now, these parameters are optimised to reduce the cost, and by the end of a sufficiently large number of steps, we get a close approximation of the transformation we need.

In [11]:
# Parameter Optimisation Loop

for step in range(steps):
  params = opt.step(lambda v: cost(v), params)
  
  weights = params[0]
  predictions = [np.sign(circuit(weights, state_id)) for state_id in range(len(target_states))]

  if not (step+1) % 10:
    print(f"step {step+1} -- accuracy: {accuracy(predictions)} , cost: {np.round(cost(params), decimals=4)}")

step 10 -- accuracy: 0.0 , cost: 9.5945
step 20 -- accuracy: 0.0 , cost: 6.2187
step 30 -- accuracy: 0.5 , cost: 3.3847
step 40 -- accuracy: 0.25 , cost: 3.0493
step 50 -- accuracy: 0.75 , cost: 2.8349
step 60 -- accuracy: 0.75 , cost: 2.6647
step 70 -- accuracy: 1.0 , cost: 2.5269
step 80 -- accuracy: 1.0 , cost: 2.4179
step 90 -- accuracy: 1.0 , cost: 2.3311
step 100 -- accuracy: 1.0 , cost: 2.2615
step 110 -- accuracy: 1.0 , cost: 2.2065
step 120 -- accuracy: 1.0 , cost: 2.1634
step 130 -- accuracy: 1.0 , cost: 2.1291
step 140 -- accuracy: 1.0 , cost: 2.1004
step 150 -- accuracy: 1.0 , cost: 2.0747
step 160 -- accuracy: 1.0 , cost: 2.0499
step 170 -- accuracy: 1.0 , cost: 2.0248
step 180 -- accuracy: 1.0 , cost: 1.9984
step 190 -- accuracy: 1.0 , cost: 1.9702
step 200 -- accuracy: 1.0 , cost: 1.9399


Finally, the circuit has been trained. And now it is time to put it to use by testing it on our input states.

In [12]:
# running the circuit on the input states with the optimised parameters
circuit_tester(params[0])

Transformations done:
(input -> output)

0111 -> 0011
target: 0011

1101 -> 0101
target: 0101

0000 -> 1010
target: 1010

1001 -> 1100
target: 1100



As seen above, the required target states are obtained as output with our variational circuit.



One interesting question as mentioned in the task was:

<b> What will happen if a different random state is given as input to our trained circuit?</b>

<b>Speculation</b>: The circuit is a combination of rotation gates (and CNOT gates) trained only to transform the old inputs states to the desired out states. On passing a different input state, the circuit will transform it to some random state (which might not necessarily be part of our previous target states). There is also a possibility that it transforms into a superposition of states. So every time we run the code, there is a chance we might even get different outputs.


In [13]:
input_states = ["1100"] # this state was not present in our previous list

In [14]:
circuit_tester(params[0], target=False)

Transformations done:
(input -> output)

1100 -> 0010


As seen above, the new input state transformed into some random state.

We can try doing this for all 4 qubit states that are not in superposition:

In [15]:
input_states = []
for i in range(16):
  bin = "{0:b}".format(i)
  bin = "0" * max(0,4 - len(bin)) + bin
  input_states.append(bin) 
print("input states:",input_states) 

input states: ['0000', '0001', '0010', '0011', '0100', '0101', '0110', '0111', '1000', '1001', '1010', '1011', '1100', '1101', '1110', '1111']


In [16]:
circuit_tester(params[0], target=False)

Transformations done:
(input -> output)

0000 -> 1010
0001 -> 1000
0010 -> 1100
0011 -> 0100
0100 -> 0100
0101 -> 1011
0110 -> 0111
0111 -> 0011
1000 -> 0101
1001 -> 1100
1010 -> 0001
1011 -> 0010
1100 -> 0010
1101 -> 0101
1110 -> 1001
1111 -> 1111
